<a href="https://colab.research.google.com/github/Ihsan1331/LLMs-Open-Course-by-Kalbe-Ahmad-Ihsan-Farhani/blob/main/1%20Hacking%20the%20HuggingFace%20Transformers%20Library/1%20Learning%20HF%20NLP/Intro_to_NLP_Ch_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Prerequisites
---
link: https://huggingface.co/learn/nlp-course/chapter4/1?fw=pt

In [1]:
!pip install datasets -q
!pip install transformers[torch] -q
!pip install accelerate -U -q
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


# 1. The Hugging Face Hub

# 2. Using pretrained models
---
Kita bisa pakai Model yang sudah di pretrained yang ada di HF

In [2]:
#bisa lewat pipeline
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

**NB Jika menggunakan pipeline, gunakan model yang sesuai dengan pretained. Jika tidak sesuai hasilny akan jelek. kecuali dilakukan pretraind**

In [3]:
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Not recommended kalau import spesifik per model (**BertTokenizer**) karena sudah ada **Auto...** yang bisa meload setiap model

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 3. Sharing Pretrained model
---
ada 3 cara buat upload pretrained ke HF:
1. push_to_hub API
2. huggingface_hub Python library
3. Interface web

In [5]:
# Login dulu ke akun HF
# bisa login dengan token
from huggingface_hub import notebook_login

notebook_login()

## 3.1 push_to_hub API

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#load dataset
raw_datasets = load_dataset("glue", "mrpc")

#pilih checkpoint
checkpoint = "bert-base-uncased"
#amil tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#tokenisasi
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

#dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [8]:
from transformers import AutoModelForSequenceClassification
# pilih model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import Trainer

trainer = Trainer(
    model, #modelnya diload pakai AutoModelForSequenceClassification
    training_args, #training args dari TrainingArguments
    train_dataset=tokenized_datasets["train"], #data train dari Datasets yang udah di tokenisasi
    eval_dataset=tokenized_datasets["validation"], # data valid dari Dataset yang udah di teokenisasi
    data_collator=data_collator, # data_collator buat dynamic_padding
    tokenizer=tokenizer, #tokenizer buat tokenisasi
)

In [10]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.517000
1000,0.307400


TrainOutput(global_step=1377, training_loss=0.3448226225055814, metrics={'train_runtime': 196.154, 'train_samples_per_second': 56.099, 'train_steps_per_second': 7.02, 'total_flos': 405324636337200.0, 'train_loss': 0.3448226225055814, 'epoch': 3.0})

Upload to HF

In [11]:
model.push_to_hub("dummy-model")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ihsan13/dummy-model/commit/465fae1eb219f7a6fb4b657df801735811c27ebb', commit_message='Upload BertForSequenceClassification', commit_description='', oid='465fae1eb219f7a6fb4b657df801735811c27ebb', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
tokenizer.push_to_hub("dummy-model")

CommitInfo(commit_url='https://huggingface.co/ihsan13/dummy-model/commit/aad67fdb3623ddc564fe2a23e4f954998bf1ca35', commit_message='Upload tokenizer', commit_description='', oid='aad67fdb3623ddc564fe2a23e4f954998bf1ca35', pr_url=None, pr_revision=None, pr_num=None)

## 3.2 huggingface_hub Python library

In [13]:
from huggingface_hub import (
    # User management
    login,
    logout,
    whoami,

    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,

    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)

In [14]:
from huggingface_hub import create_repo

create_repo("dummy-model-huggingface_hub")

RepoUrl('https://huggingface.co/ihsan13/dummy-model-huggingface_hub', endpoint='https://huggingface.co', repo_type='model', repo_id='ihsan13/dummy-model-huggingface_hub')